# 1、字符串输出解析器 StrOutputParser

In [6]:
# 1、获取大模型
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser

import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")
# print(type(response))   # AIMessage

# 3、如何获取一个字符串的输出结果呢？
# # 方式1：自己调用输出结果的content
# print(response.content)

# 方式2：使用StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  # <class 'str'>
print(str_response)


<class 'str'>
大语言模型是指一种能够生成自然语言文本的机器学习模型，这些模型通常基于深层神经网络结构，如变换器（Transformer）架构，具有大量的参数（通常超过百万级甚至亿级参数）。大语言模型通过对大量文本数据进行训练，可以理解和生成与人类相似的自然语言文本，具备强大的语言理解和生成能力。

大语言模型能够完成的任务范围广泛，包括但不限于文本补全、机器翻译、情感分析、问答系统、文本分类、摘要生成等。在近年来自然语言处理领域取得了重要进展，成为诸多文本生成任务的有力工具。代表性的大语言模型包括但不限于：阿里云的通义千问、谷歌的LaMDA、百度的文心一言、阿里云的通义万相、百度ERNIE等。


# 2、JsonOutputParser : Json输出解析器

方式1：

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 正确的：
prompt = chat_prompt_template.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# 错误的：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？"})

response = chat_model.invoke(prompt)
# print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

{'q': '人工智能用英文怎么说？', 'a': '人工智能用英文说为Artificial Intelligence (AI)'}


方式2：

举例1：

In [9]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


举例2：

In [10]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response)

json_result = parser.invoke(response)
print(json_result)

content='```json\n{\n  "joke": "为什么电脑经常生病？因为窗户（Windows）总是开着！"\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 52, 'total_tokens': 78, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': '', 'id': '019bf9036ae9f033638cd9c64685e96b', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bf903-6d88-71c3-8ff1-7690fd2a1514-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 52, 'output_tokens': 26, 'total_tokens': 78, 'input_token_details': {}, 'output_token_details': {}}
{'joke': '为什么电脑经常生病？因为窗户（Windows）总是开着！'}


知识的拓展： |

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 写法1：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
#
# response = chat_model.invoke(prompt)
#
# json_result = parser.invoke(response)
# print(json_result)

# 写法2：
chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"}
    )
print(json_result1)

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence (AI)'}


针对于举例2

In [14]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 写法1：
# prompt = prompt_template.invoke(input={"question":joke_query})
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)

{'joke': '为什么电脑经常生病？因为窗户（windings）总是开着！'}


# 3、XMLOutputParser XML输出解析器的使用

举例1：自己在提示词模板中写明使用XML格式

In [1]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

actor_query = "周星驰的简短电影记录"
response = chat_model.invoke(f"请生成{actor_query}，将影片附在<movie></movie>标签中")

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
当然可以。这里有一个简短的周星驰电影记录供您参考：

```html
<movie>
    <title>大话西游之月光宝盒</title>
    <year>1995</year>
    <director>周星驰</director>
    <actor>
        <name>周星驰</name>
        <name>朱茵</name>
    </actor>
    <genre>奇幻、喜剧</genre>
    <description>这部影片讲述了迟来的唐僧从月宫盗取月光宝盒，由此引发了蛛丝马迹的爱情、误会交织的故事。</description>
</movie>
```

这个示例呈现了《大话西游之月光宝盒》的部分基本信息，包括电影名称、主演和简要描述等。如果有更多电影需要记录，可以按照这样的结构进行添加。


举例2：

In [2]:
from langchain_core.output_parsers.xml import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


使用parser.get_format_instructions()结构实现：

In [3]:
# 1.导入相关包
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 2. 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 3.测试模型的xml解析效果
actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

# 4.定义XMLOutputParser对象
parser = XMLOutputParser()

# 5. 生成提示词模板
prompt_template1 = PromptTemplate.from_template(
    template="用户的问题：{query}\n使用的格式：{format_instructions}"
)

prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())


response = chat_model.invoke(prompt_template2.invoke(input={"query": actor_query}))
print(response.content)

<TomHanks>
   <影片记录>
      <电影名称>阿甘正传</电影名称>
      <上映年份>1994</上映年份>
      <电影名称>拯救大兵瑞恩</电影名称>
      <上映年份>1998</上映年份>
      <电影名称>欲望城市</电影名称>
      <上映年份>2008</上映年份>
   </影片记录>
</TomHanks>


In [4]:
xml_result = parser.invoke(response)
print(xml_result)

{'TomHanks': [{'影片记录': [{'电影名称': '阿甘正传'}, {'上映年份': '1994'}, {'电影名称': '拯救大兵瑞恩'}, {'上映年份': '1998'}, {'电影名称': '欲望城市'}, {'上映年份': '2008'}]}]}


# 4、列表解析器 CommaSeparatedListOutputParser

举例1：

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


举例2：

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
    "format_instructions": output_parser.get_format_instructions()
    })

# 提示模板与输出解析器传递输出
# chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

['科幻电影', '爱情电影', '动作电影', '喜剧电影', '悬疑电影']
<class 'list'>


# 5、日期解析器 DatetimeOutputParser（太老了，已经被移除）

举例1：

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from datetime import datetime

# 1. 定义一个只包含日期的结构
class DateResponse(BaseModel):
    date_value: datetime = Field(description="事件发生的日期，必须严格使用 YYYY-MM-DD 格式 (ISO 8601)")

# 2. 使用 PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=DateResponse)

# 3. 获取格式指令
format_instructions = output_parser.get_format_instructions()

print("=== 指令内容 ===")
print(format_instructions)

=== 指令内容 ===
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date_value": {"description": "事件发生的日期，必须严格使用 YYYY-MM-DD 格式 (ISO 8601)", "format": "date-time", "title": "Date Value", "type": "string"}}, "required": ["date_value"]}
```


举例2：


In [10]:
import os
from datetime import datetime
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 1. 定义你想要的数据结构
class TimeResponse(BaseModel):
    date_value: datetime = Field(description="The date of the event")

# 2. 初始化模型
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 3. 使用 v1.x 的核心方法 .with_structured_output()
structured_llm = llm.with_structured_output(TimeResponse)

# 4. 直接调用
try:
    response = structured_llm.invoke("中华人民共和国是什么时候成立的？")
    print(f"日期: {response.date_value}") # 直接拿到 datetime 对象
    print(f"格式化: {response.date_value.strftime('%Y-%m-%d')}")
    print(f"类型: {type(response.date_value)}")
except Exception as e:
    print(e)

日期: 1949-10-01 00:00:00+00:00
格式化: 1949-10-01
类型: <class 'datetime.datetime'>
